In [13]:
%matplotlib inline
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn import datasets
from sklearn.metrics import make_scorer,accuracy_score,recall_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import joblib

import warnings
warnings.filterwarnings("ignore")

# #生成数据集
# data_X, data_Y = datasets.make_classification(
#     n_samples=5000,n_features=16,n_informative=12,n_redundant=0,n_classes=4,n_clusters_per_class=1,random_state=10)
#分割训练集和测试集
# train_X, test_X, train_y, test_y=train_test_split(data_X,data_Y,test_size=0.3,random_state=10)

#输入原始数据
data=np.loadtxt('jm_mode3.csv',dtype=int,delimiter=',',unpack=False)
X=data[:,:17]
y=data[:,17]
train_X, test_X, train_y, test_y = train_test_split(X,y,test_size=0.3,random_state=10)

#标准化特征数据
ss=StandardScaler()
train_X_new=ss.fit_transform(train_X)
test_X_new=ss.transform(test_X)


#实例化一个随机森林
rfc = RandomForestClassifier(oob_score=True, random_state=10)
rfc.fit(train_X_new,train_y)
print ('初始袋外得分：',rfc.oob_score_)


初始袋外得分： 0.9983948635634029


In [7]:
# #pca降维
# oob_score=[]
# for i in range(3,16):
#     pca = PCA(n_components=i, random_state=10)
#     pca.fit(train_X_new)
#     #print (pca.explained_variance_ratio_)
#     #print (pca.explained_variance_)

#     #pca降维后的模型得分
#     train_X_new2=pca.fit_transform(train_X_new)
#     test_X_new2=pca.transform(test_X_new)
#     rfc.fit(train_X_new2,train_y)
#     oob_score.append(rfc.oob_score_)
#     print('维数：',i,'\t袋外得分：',rfc.oob_score_)
# print('最佳维数：',oob_score.index(max(oob_score))+3,'\t袋外得分：',max(oob_score))

In [17]:
# #最佳降维数据集
# pca = PCA(n_components=5, random_state=10)
# pca.fit(train_X_new)
# train_X_new2=pca.fit_transform(train_X_new)
# test_X_new2=pca.transform(test_X_new)

In [14]:
#最优决策树数目和最大深度
param_test1 = {'n_estimators':range(100,250,10),'max_depth':range(3,15,1)}
gsearch1 = GridSearchCV(estimator = RandomForestClassifier(random_state=10),
                        param_grid = param_test1, scoring='recall_micro',cv=5,n_jobs=-1)
gsearch1.fit(train_X_new,train_y)
pd.DataFrame(gsearch1.cv_results_).loc[:, ['params','mean_test_score']], gsearch1.best_params_, gsearch1.best_score_

(                                     params  mean_test_score
 0     {'max_depth': 3, 'n_estimators': 100}           0.9952
 1     {'max_depth': 3, 'n_estimators': 110}           0.9936
 2     {'max_depth': 3, 'n_estimators': 120}           0.9952
 3     {'max_depth': 3, 'n_estimators': 130}           0.9936
 4     {'max_depth': 3, 'n_estimators': 140}           0.9936
 ..                                      ...              ...
 175  {'max_depth': 14, 'n_estimators': 200}           0.9984
 176  {'max_depth': 14, 'n_estimators': 210}           1.0000
 177  {'max_depth': 14, 'n_estimators': 220}           1.0000
 178  {'max_depth': 14, 'n_estimators': 230}           1.0000
 179  {'max_depth': 14, 'n_estimators': 240}           1.0000
 
 [180 rows x 2 columns], {'max_depth': 6, 'n_estimators': 100}, 1.0)

In [14]:
#最大特征数
param_test2 = {'max_features':range(1,16)}
gsearch2 = GridSearchCV(estimator = RandomForestClassifier(n_estimators= 60,max_depth=10,random_state=10),
                        param_grid = param_test2, scoring='recall_micro',cv=5,n_jobs=-1)
gsearch2.fit(train_X_new,train_y)
pd.DataFrame(gsearch2.cv_results_).loc[:, ['params','mean_test_score']], gsearch2.best_params_, gsearch2.best_score_

(                  params  mean_test_score
 0    {'max_features': 1}         0.893333
 1    {'max_features': 2}         0.920000
 2    {'max_features': 3}         0.920000
 3    {'max_features': 4}         0.926667
 4    {'max_features': 5}         0.933333
 5    {'max_features': 6}         0.933333
 6    {'max_features': 7}         0.933333
 7    {'max_features': 8}         0.920000
 8    {'max_features': 9}         0.940000
 9   {'max_features': 10}         0.940000
 10  {'max_features': 11}         0.933333
 11  {'max_features': 12}         0.926667
 12  {'max_features': 13}         0.926667
 13  {'max_features': 14}         0.920000
 14  {'max_features': 15}         0.926667,
 {'max_features': 9},
 0.9400000000000001)

In [15]:
# 模型评估
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

rfc = RandomForestClassifier(n_estimators= 100,max_depth=6,oob_score=True, random_state=10)
rfc.fit(train_X_new,train_y)
print('随机森林袋外得分：',rfc.oob_score_)

#对最终模型进行评估
test_y_pred=rfc.predict(test_X_new)
confusion_mat = confusion_matrix(test_y,test_y_pred)
#print(confusion_mat) #看看混淆矩阵长啥样
print('*'*53)
print(classification_report(test_y,test_y_pred))
print('*'*53)

随机森林袋外得分： 0.9983948635634029
*****************************************************
              precision    recall  f1-score   support

           1       1.00      1.00      1.00       220
           2       1.00      1.00      1.00        25
           3       1.00      1.00      1.00        23

    accuracy                           1.00       268
   macro avg       1.00      1.00      1.00       268
weighted avg       1.00      1.00      1.00       268

*****************************************************


In [16]:
# 打包
joblib.dump(rfc, "rfc_mode3.m")

['rfc_mode3.m']

In [29]:
#第一次调参后的检验，可不加
rfc = RandomForestClassifier(n_estimators= 220,oob_score=True,random_state=10)
rfc.fit(train_X_new,train_y)
print('袋外得分：',rfc.oob_score_)

袋外得分： 0.9322857142857143


In [28]:
#最大深度和最小分割样本数
param_test2 = {'max_depth':range(11,20,2), 'min_samples_split':range(2,11,2)}
#Scoring = ['recall_micro','f1_micro']
gsearch2 = GridSearchCV(estimator = RandomForestClassifier(n_estimators= 220,random_state=10),
                        param_grid = param_test2, scoring='recall_micro',cv=5,n_jobs=-1)
gsearch2.fit(train_X_new,train_y)
pd.DataFrame(gsearch2.cv_results_).loc[:, ['params','mean_test_score']], gsearch2.best_params_, gsearch2.best_score_

(                                        params  mean_test_score
 0    {'max_depth': 11, 'min_samples_split': 2}         0.927143
 1    {'max_depth': 11, 'min_samples_split': 4}         0.924000
 2    {'max_depth': 11, 'min_samples_split': 6}         0.928000
 3    {'max_depth': 11, 'min_samples_split': 8}         0.921714
 4   {'max_depth': 11, 'min_samples_split': 10}         0.922286
 5    {'max_depth': 13, 'min_samples_split': 2}         0.931714
 6    {'max_depth': 13, 'min_samples_split': 4}         0.925429
 7    {'max_depth': 13, 'min_samples_split': 6}         0.928571
 8    {'max_depth': 13, 'min_samples_split': 8}         0.927429
 9   {'max_depth': 13, 'min_samples_split': 10}         0.924571
 10   {'max_depth': 15, 'min_samples_split': 2}         0.930286
 11   {'max_depth': 15, 'min_samples_split': 4}         0.927429
 12   {'max_depth': 15, 'min_samples_split': 6}         0.928571
 13   {'max_depth': 15, 'min_samples_split': 8}         0.927429
 14  {'max_depth': 15, 'm

In [30]:
#最小分割样本数与最小叶节点样本数
param_test3 = {'min_samples_leaf':range(1,10,2), 'min_samples_split':range(2,11,2)}
gsearch3 = GridSearchCV(estimator = RandomForestClassifier(n_estimators= 220,max_depth=13,random_state=10),
                        param_grid = param_test3, scoring='recall_micro',cv=5,n_jobs=-1)
gsearch3.fit(train_X_new,train_y)
pd.DataFrame(gsearch3.cv_results_).loc[:, ['params','mean_test_score']], gsearch3.best_params_, gsearch3.best_score_

(                                              params  mean_test_score
 0    {'min_samples_leaf': 1, 'min_samples_split': 2}         0.929143
 1    {'min_samples_leaf': 1, 'min_samples_split': 4}         0.924000
 2    {'min_samples_leaf': 1, 'min_samples_split': 6}         0.927143
 3    {'min_samples_leaf': 1, 'min_samples_split': 8}         0.928571
 4   {'min_samples_leaf': 1, 'min_samples_split': 10}         0.924857
 5    {'min_samples_leaf': 3, 'min_samples_split': 2}         0.923714
 6    {'min_samples_leaf': 3, 'min_samples_split': 4}         0.923714
 7    {'min_samples_leaf': 3, 'min_samples_split': 6}         0.923714
 8    {'min_samples_leaf': 3, 'min_samples_split': 8}         0.923143
 9   {'min_samples_leaf': 3, 'min_samples_split': 10}         0.922286
 10   {'min_samples_leaf': 5, 'min_samples_split': 2}         0.915143
 11   {'min_samples_leaf': 5, 'min_samples_split': 4}         0.915143
 12   {'min_samples_leaf': 5, 'min_samples_split': 6}         0.915143
 13   

In [31]:
#最大特征数
param_test4 = {'max_features':range(1,16)}
gsearch4 = GridSearchCV(estimator = RandomForestClassifier(n_estimators= 220,max_depth=13,min_samples_leaf=1,min_samples_split=2,random_state=10),
                        param_grid = param_test4, scoring='recall_micro',cv=5,n_jobs=-1)
gsearch4.fit(train_X_new,train_y)
pd.DataFrame(gsearch4.cv_results_).loc[:, ['params','mean_test_score']], gsearch4.best_params_, gsearch4.best_score_

(                  params  mean_test_score
 0    {'max_features': 1}         0.903143
 1    {'max_features': 2}         0.923429
 2    {'max_features': 3}         0.926000
 3    {'max_features': 4}         0.929143
 4    {'max_features': 5}         0.932286
 5    {'max_features': 6}         0.931714
 6    {'max_features': 7}         0.930286
 7    {'max_features': 8}         0.929714
 8    {'max_features': 9}         0.926000
 9   {'max_features': 10}         0.930571
 10  {'max_features': 11}         0.926857
 11  {'max_features': 12}         0.924000
 12  {'max_features': 13}         0.922286
 13  {'max_features': 14}         0.923143
 14  {'max_features': 15}         0.920286,
 {'max_features': 5},
 0.9322857142857142)

In [32]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

gnb = GaussianNB()

# #支持向量机调参
# param_test5 = {'C':np.arange(1,2.5,0.2),'gamma':np.arange(0.02,0.026,0.001)}
# gsearch5 = GridSearchCV(estimator = SVC(kernel='rbf',decision_function_shape='ovr'),
#                         param_grid = param_test5, scoring='recall_micro',cv=5,n_jobs=-1)
# gsearch5.fit(train_X_new,train_y)
# print(pd.DataFrame(gsearch5.cv_results_).loc[:, ['params','mean_test_score']], gsearch5.best_params_, gsearch5.best_score_)
svc=SVC(C=1.8,gamma=0.021,kernel='rbf',decision_function_shape='ovr')

# #逻辑回归调参
# param_test6 = {'C':np.arange(0.05,0.5,0.01),'penalty':['l1','l2']}
# gsearch6 = GridSearchCV(estimator = LogisticRegression(solver = "liblinear"),
#                         param_grid = param_test6, scoring='recall_micro',cv=5,n_jobs=-1)
# gsearch6.fit(train_X_new,train_y)
# print(pd.DataFrame(gsearch6.cv_results_).loc[:, ['params','mean_test_score']], gsearch6.best_params_, gsearch6.best_score_)
lr=LogisticRegression(solver = "liblinear",C=0.11,penalty='l1')

# #knn调参
# param_test7 = {'n_neighbors':range(15,30,1),'p':range(1,6,1)}
# gsearch7 = GridSearchCV(estimator = KNeighborsClassifier(weights='distance'),
#                         param_grid = param_test7, scoring='recall_micro',cv=5,n_jobs=-1)
# gsearch7.fit(train_X_new,train_y)
# print(pd.DataFrame(gsearch7.cv_results_).loc[:, ['params','mean_test_score']], gsearch7.best_params_, gsearch7.best_score_)
knn=KNeighborsClassifier(n_neighbors=25,weights='distance',p=2)

knn.fit(train_X_new,train_y)
lr.fit(train_X_new,train_y)
gnb.fit(train_X_new,train_y)
svc.fit(train_X_new,train_y)
print('knn得分',knn.score(test_X_new,test_y))
print('逻辑回归得分',lr.score(test_X_new,test_y))
print('朴素贝叶斯得分',gnb.score(test_X_new,test_y))
test_y_pred=svc.predict(test_X_new)
print('支持向量机得分:\n',classification_report(test_y,test_y_pred))
print('*'*50)

rfc = RandomForestClassifier(n_estimators= 220, max_depth=13, min_samples_split=2,
                                  min_samples_leaf=1,max_features=5 ,oob_score=True, random_state=10)
rfc.fit(train_X_new,train_y)
print('随机森林袋外得分：',rfc.oob_score_)
print('*'*50)
#对最终模型进行评估
test_y_pred=rfc.predict(test_X_new)
confusion_mat = confusion_matrix(test_y,test_y_pred)
print(confusion_mat) #看看混淆矩阵长啥样
print('*'*50)
print(classification_report(test_y,test_y_pred))

knn得分 0.9286666666666666
逻辑回归得分 0.842
朴素贝叶斯得分 0.8186666666666667
支持向量机得分:
               precision    recall  f1-score   support

           0       0.95      0.97      0.96       383
           1       0.96      0.93      0.95       383
           2       0.91      0.92      0.92       368
           3       0.92      0.91      0.91       366

    accuracy                           0.93      1500
   macro avg       0.93      0.93      0.93      1500
weighted avg       0.93      0.93      0.93      1500

**************************************************
随机森林袋外得分： 0.9305714285714286
**************************************************
[[358   5  14   6]
 [  6 358   8  11]
 [ 15   4 337  12]
 [  8   7  10 341]]
**************************************************
              precision    recall  f1-score   support

           0       0.93      0.93      0.93       383
           1       0.96      0.93      0.95       383
           2       0.91      0.92      0.91       368
           3  